<!-- # Processing the final dataset
 -->


# Export data as CSV

In [2]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year


## Main Model Dataset

### Age, Biomass

In [ ]:
# Fire and Land Use
age = ee.Image(f"{data_folder}/mapbiomas_{last_year}")

ESA_CCI = ee.Image(f"projects/sat-io/open-datasets/ESA/ESA_CCI_AGB/CCI_BIOMASS_100m_AGB_{last_year}_v51").select("AGB").rename("biomass")

fire = (ee.Image("projects/mapbiomas-public/assets/brazil/fire/collection3/mapbiomas_fire_collection3_annual_burned_coverage_v1")
    .select([f"burned_coverage_{year}" for year in config.range_1985_2020])
    .byte()
    .rename([str(year) for year in config.range_1985_2020])
    .gt(0)
    .reduce('sum').rename("num_fires")).unmask(0)

floodable_forests = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
        .select(f"classification_{last_year}").eq(6)).rename("floodable_forests")

secondary_area = ee.Image(f"{data_folder}/secondary_area")
pasture_area = ee.Image(f"{data_folder}/pasture_area")


### Surrounding Landscape

In [4]:
quarters_ecoreg_biomass = ee.Image("projects/amazon-forest-regrowth/assets/quarters_ecoreg_biomass")
distance_deep_forest = ee.Image(f"{data_folder}/distance_deep_forest").rename("dist")
sur_cover = ee.Image(f"{data_folder}/sur_cover")

distance_gt_1000 = distance_deep_forest.gt(1000).selfMask()
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
mature_biomass_10k = ee.Image(f"{data_folder}/mature_biomass_10k")
nearest_mature = ee.Image(f"{data_folder}/nearest_mature")

### Environmental

In [5]:
categorical = ee.Image(f"{data_folder}/categorical")

topography = ee.Image("CSP/ERGo/1_0/Global/ALOS_landforms").rename("topography") # 90m resolution

soil = ee.Image(f"{data_folder}/soilgrids")

terraclim = ee.Image(f"{data_folder}/terraclim_1958_2019")

In [6]:
# Load each scenario
historical = ee.Image(f"{data_folder}/CMIP6_historical")
ssp126 = ee.Image(f"{data_folder}/CMIP6_ssp126")
ssp245 = ee.Image(f"{data_folder}/CMIP6_ssp245")
ssp585 = ee.Image(f"{data_folder}/CMIP6_ssp585")

CMIP6_means = ee.Image(f"{data_folder}/CMIP6_means")

### Export CSV


- 1 secondary forest pixel per 1km2
- 1 pasture pixel per 1km2

In [ ]:
# create_grid(age.gt(0).unmask(0), "atlantic", cell_size = 1000, file_name = "secondary")
# create_grid(age.gt(0).unmask(0), "amazon", cell_size = 1000, file_name = "secondary")
# create_grid(age.gt(0).unmask(0), "amazon", cell_size = 10000, file_name = "secondary1")
# create_grid(pastureland, "amazon", cell_size = 1000, file_name = "pastureland")

# make one grid with all pixels that are NOT mature forests for nearest mature forest estimates
# non_mature_forests = mature_biomass_10k.eq(0).unmask(1).selfMask().rename("non_mature_forests")
# create_grid(non_mature_forests, "amazon", cell_size = 10000, file_name = "non_mature_forests")

In [ ]:

def create_grid(image, region_name, cell_size = 10000, file_name = None):
    
    config = ProjectConfig()
    biomes = ee.Image(f"{config.data_folder}/categorical").select("biome")

    pixels_to_sample = biomes.updateMask(image)

    if region_name == "amazon":
        biome_index = 1
    elif region_name == "atlantic":
        biome_index = 4

    roi = ee.FeatureCollection(f"{config.data_folder}/raw/biomes_br").filterMetadata('CD_Bioma', 'equals', biome_index).geometry()

    # First, sample locations based only on the age band
    grid = geemap.create_grid(roi, cell_size, biomes.projection())

    # Function to sample one point per valid cell
    def sample_cell(cell):
        sampled_fc = pixels_to_sample.stratifiedSample(
            numPoints = 1,
            classBand = 'biome',
            region = cell.geometry(),
            scale = pixels_to_sample.projection().nominalScale(),
            geometries = True,
            dropNulls = True,
            tileScale = 12
        )

        # Only return a feature if we found one
        return ee.Feature(ee.Algorithms.If(
            sampled_fc.size().gt(0),
            sampled_fc.first(),
            # Return a placeholder that we can filter out later
            ee.Feature(ee.Geometry.Point([0, 0])).set('is_null', True)
        ))

    samples = grid.map(sample_cell)

    # Filter out placeholder features before exporting
    samples = samples.filter(ee.Filter.notEquals('is_null', True))

    if file_name is None:
        return samples
    else:
        export_name = f"grid_{cell_size//1000}k_{region_name}_{file_name}"

        export_task = ee.batch.Export.table.toAsset(
            collection = samples,
            description = export_name,
            assetId = f"{config.data_folder}/{export_name}"
        )
        export_task.start()


# grid = create_grid(age.gt(0).unmask(0), "amazon", cell_size=1000000)


biomes = ee.Image(f"{config.data_folder}/categorical").select("biome")
biomes = biomes.updateMask(age.gt(0).unmask(0))
grid = geemap.create_grid(biomes.geometry(), 1000000, biomes.projection())

map = geemap.Map()
map.addLayer(biomes, {}, "Biomes")
map.addLayer(grid, {}, "Grid Secondary")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [104]:
biomes.projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:4326',
 'transform': [0.004491576420597608,
  0,
  -73.9852468000838,
  0,
  -0.004491576420597608,
  5.2731107177815915]}

In [10]:
# make one unified grid for all pastureland and secondary forests
grid_secondary = ee.FeatureCollection(f"{data_folder}/grid_1k_amazon_secondary")
grid_pastureland = ee.FeatureCollection(f"{data_folder}/grid_1k_amazon_pastureland")

# merge both grids
grid_1k_amazon_non_mature_forests = grid_secondary.merge(grid_pastureland)

task = ee.batch.Export.table.toAsset(
    collection = grid_1k_amazon_non_mature_forests,
    description = "grid_1k_amazon_non_mature_forests",
    assetId = "projects/amazon-forest-regrowth/assets/grid_1k_amazon_non_mature_forests",
)
# task.start()


In [102]:
# Define reprojection function
def reproject_mean(img):
    return img.reduceResolution(ee.Reducer.mean(), maxPixels=65536).reproject(age.projection())

def reproject_first(img):
    return img.reduceResolution(ee.Reducer.first(), maxPixels=65536).reproject(age.projection())

# Reproject continuous variables using mean
continuous_vars = [
    ESA_CCI.rename("biomass"),
    nearest_mature.rename("nearest_mature"),
    soil,
    terraclim.select(["mean_soil", "mean_vpd", "mean_temp", "mean_def", 
                      "mean_srad", "mean_pr", "mean_aet", "mean_pdsi"])
]

# Reproject categorical variables using first
categorical_vars = [
    fire,
    quarters_ecoreg_biomass,
    categorical,
    topography
]

# Create unified dataset by directly combining bands
unified_data = ee.Image.cat([
    age.rename('age'),
    floodable_forests.rename('floodable_forests'),
    secondary_area.rename('secondary_area'),
    distance_deep_forest.rename('distance_deep_forest'),
    sur_cover.rename('sur_cover'),
    *[reproject_mean(img) for img in continuous_vars],  # Expand list of reprojected images
    *[reproject_first(img) for img in categorical_vars],  # Expand list of reprojected images
    ee.Image.pixelLonLat().rename(['lon', 'lat'])
])


Since processing gets too heavy if we try to extract each point at a time for the entire Amazon at 1km resolution, we do it in chunks:

In [ ]:


def export_csv(name, image, grid_size, n_chunks = 1):

    properties_to_export = image.bandNames().getInfo()

    # Get FeatureCollection size and calculate chunk size
    grid = ee.FeatureCollection(f"{data_folder}/grid_{grid_size}k_amazon_{name}")
    total_features = grid.size().getInfo()
    chunk_size = int(total_features * 1/n_chunks)

    # Sample points individually
    def sample_point(feature):
        sampled = image.reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=feature.geometry(),
            scale=30,
            crs='EPSG:4326',
            bestEffort=False,
            tileScale=16
        )
        return ee.Feature(feature.geometry(), sampled)

    def process_chunk(chunk_index):
        start = chunk_index * chunk_size
        chunk = grid.toList(chunk_size, start)
        selected_pixels = ee.FeatureCollection(chunk)

        unified_fc = selected_pixels.map(sample_point)

        # task = ee.batch.Export.table.toDrive(
        #     collection = unified_fc,
        #     description = f"grid_{grid_size}k_amazon_{name}_{chunk_index}",
        #     fileFormat = "CSV",
        #     selectors = [p for p in properties_to_export if p not in ['.geo', 'system:index']]
        # )
        # task.start()

        task = ee.batch.Export.table.toAsset(
            collection = unified_fc,
            description = f"{grid_size}k_amazon_{name}",
            assetId = f"projects/amazon-forest-regrowth/assets/grids/amazon_{name}_{grid_size}k_ESA_CCI",

        )
        task.start()

    # for i in range(n_chunks):
    #     if i*chunk_size < total_features:
    #         process_chunk(i)

    i = 0
    if i*chunk_size < total_features:
        process_chunk(i)

# export_csv("pastureland", unified_data, 1, n_chunks = 60)
export_csv("secondary", unified_data, 1, n_chunks = 30)
export_csv("secondary", unified_data, 10, n_chunks = 30)

# the scale here will need to be the same as the scale of the sampled image.
# test with just ESA and sample to 100m

In [80]:
amazon_secondary_10k = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/grids/amazon_secondary_10k_ESA_CCI")
amazon_secondary_1k = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/grids/amazon_secondary_1k_ESA_CCI")


map = geemap.Map(center=[-3.5, -60.24], zoom=6)
map.addLayer(ESA_CCI, {'min':50, 'max':200, 'palette':['blue', 'red']}, "ESA_CCI", opacity=0.4)
map.addLayer(unified_data, {}, "unified_data", opacity=0.1)
map.addLayer(amazon_secondary_10k, {}, "10k_amazon_secondary", opacity=0.4)
map.addLayer(amazon_secondary_1k, {}, "1k_amazon_secondary", opacity=0.4)
map

Map(center=[-3.5, -60.24], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

## Field Data

In [ ]:
field_data = ee.FeatureCollection(f"{data_folder}/field_biomass")

unified_fc = unified_data.reduceRegions(field_data, ee.Reducer.first(), 30)

# Export task to Google Drive
task = ee.batch.Export.table.toDrive(
    collection = unified_fc,
    description = 'unified_field',
    fileFormat = "CSV"
)
# task.start()

## Different land use aggregations


In [ ]:
suffixes = [
    "aggregated_all",
    "non_aggregated_all",
    "non_aggregated_10yr",
    "non_aggregated_5yr"
]

land_use_list = [
    ee.Image(f"{data_folder}/land_use_{suffix}")
    for suffix in suffixes
]

# Loop through each land use image
for land_use_image, suffix in zip(land_use_list, suffixes):

    selected_pixels = ee.FeatureCollection(f"{data_folder}/grid_10k_amazon_secondary")

    unified_fc = land_use_image.reduceRegions(selected_pixels, ee.Reducer.first(), 30)

    properties_to_export = land_use_image.bandNames().getInfo()

    task = ee.batch.Export.table.toDrive(
            collection = unified_fc,
            description = "unified_land_use_" + suffix,
            fileFormat = "CSV",
            selectors = properties_to_export
        )
    task.start()


In [34]:
selected_pixels = ee.FeatureCollection(f"{data_folder}/grid_10k_amazon_secondary")
# WITH BUFFER

mature_biomass_10k = ee.Image(f"{data_folder}/mature_biomass_10k").rename("mature_biomass_10k")

selected_pixels = distance_deep_forest.reduceRegions(
    collection = selected_pixels,
    reducer = ee.Reducer.first(),
    scale = 10000
)

def buffer_feature(feature):
    distance = feature.getNumber('first').add(10000)
    buffer = feature.geometry().buffer(distance)
    return feature.setGeometry(buffer)

# Buffer each point to reach the nearest pixel
buffered_features = selected_pixels.map(buffer_feature)

# Extract the biomass value for each buffered region
# This will get the value from nearest valid pixel
nearest_mature = mature_biomass_10k.reduceRegions(
    collection=buffered_features,
    reducer=ee.Reducer.firstNonNull(),
    scale=10000
).map(lambda feature: feature.centroid())


unified_data = ESA_CCI.addBands(age)

unified_fc = unified_data.reduceRegions(nearest_mature, ee.Reducer.first(), 30)

# Export task to Google Drive
task = ee.batch.Export.table.toDrive(
    collection = unified_fc,
    description = 'buffered_nearest_mature',
    fileFormat = "CSV"
)
task.start()

# task = ee.batch.Export.table.toAsset(
#     collection = unified_fc,
#     description = 'unified_fc',
#     assetId = f"{data_folder}/unified_fc"
# )
# task.start()